# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [1]:
import torch
import torch.nn as nn

In [2]:
print(torch.backends.mps.is_available())

True


In [3]:
if torch.backends.mps.is_available():
    device = torch.device("mps") # Apple GPU
    print("Using MPS device")
else:
    device = torch.device("cpu") # Defaults to CPU
    print("MPS device not found, using CPU")

# Example: Move a tensor or model to the MPS device
x = torch.ones(5, device=device)
class TwoBranch(nn.Module):
    def __init__(self):
        super().__init__()
        model = nn.LSTM(input_size=3, hidden_size=4, batch_first=False, num_layers=3)
        self.left = model
        self.right = model
        self.combine = nn.Linear(8, 8)

    def forward(self, x_left, x_right):
        _, (l_h, _) = self.left(x_left)
        _, (r_h, _) = self.right(x_right)

        l = l_h[-1]  # last layer, shape: (batch, hidden_size)
        r = r_h[-1]

        cat = torch.cat([l, r], dim=-1)  # shape: (batch, 10)
        return self.combine(cat)          # shape: (batch, 10) -> Linear(10,1) -> (batch,1)
class BigModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.branch = TwoBranch()
        self.rest = nn.Sequential(
            nn.Linear(8, 2),
            nn.ReLU(),
            nn.Linear(2, 1),
            nn.Sigmoid()
        )

    def forward(self, x_left, x_right):
        x = self.branch(x_left, x_right)
        return self.rest(x)
model2=BigModel()
model2.to(device)



Using MPS device


BigModel(
  (branch): TwoBranch(
    (left): LSTM(3, 4, num_layers=3)
    (right): LSTM(3, 4, num_layers=3)
    (combine): Linear(in_features=8, out_features=8, bias=True)
  )
  (rest): Sequential(
    (0): Linear(in_features=8, out_features=2, bias=True)
    (1): ReLU()
    (2): Linear(in_features=2, out_features=1, bias=True)
    (3): Sigmoid()
  )
)

In [4]:
"""
batch = 32
seq_len = 64

x_left  = torch.randn(seq_len, batch, 3, device=device)
x_right = torch.randn(seq_len, batch, 3, device=device)

# labels depend on problem:
y = torch.randn(batch, 1, device=device)  # regression
"""

'\nbatch = 32\nseq_len = 64\n\nx_left  = torch.randn(seq_len, batch, 3, device=device)\nx_right = torch.randn(seq_len, batch, 3, device=device)\n\n# labels depend on problem:\ny = torch.randn(batch, 1, device=device)  # regression\n'

In [5]:
import pandas as pd
import glob
files = glob.glob("data/*.json")
people = []
for file in files:
    df = pd.read_json(file)
    drawings = []
    for drawing in df.values:
        points = drawing[0][0]["points"]
        triples = [(point["x"], point["y"], point["time"]) for point in points]
        drawings.append(triples)
    people.append(drawings)


data = people


In [6]:
import json
from pathlib import Path

out_path = Path("output/data.json")

out_path.write_text(json.dumps(data, indent=2))

485185

In [7]:
out_path = Path("output/data.json")
loaded = json.load(out_path.open())

In [8]:
import numpy as np

training: list[list[list[tuple[int, int, int]]]] = loaded[0:3]
pairedDeep = [[torch.tensor(drawing,device=device) for drawing in drawings] for drawings in training]
paired: list[torch.Tensor] = []
personIDs: list[int] = []
for personID, person in enumerate(pairedDeep):
    a = [personID]*len(person)
    personIDs.extend(a)
    paired.extend(person)
personIDs = np.array(personIDs)
lengths = torch.tensor([len(s) for s in paired])
def packPad(pairs: list[torch.Tensor], lengths: torch.Tensor, indeces):
    padded = nn.utils.rnn.pad_sequence([pairs[i] for i in indeces], batch_first=True)
    padded.to(device)
    lengths = lengths[indeces]
    packed = nn.utils.rnn.pack_padded_sequence(
        padded,
        lengths,
        batch_first=True,
        enforce_sorted=False
    ).float()
    packed.to(device)
    return packed

permuteLeft = np.random.permutation(len(paired))
permuteRight = np.random.permutation(len(paired))
pairedNP = paired
lengthsNP = lengths
left = packPad(pairedNP, lengthsNP, permuteLeft)
right = packPad(pairedNP, lengthsNP, permuteRight)

y = (torch.tensor(personIDs[permuteLeft] == personIDs[permuteRight])
     .reshape((-1,1))
     .to(device, dtype=torch.float32))

In [9]:
batch_size = 32

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=0.001)

for epoch in range(100):
    optimizer.zero_grad()
    epoch_loss = 0.0

    # iterate over batches
    for i in range(0, len(permuteLeft), batch_size):
        idx_left = permuteLeft[i:i + batch_size]
        idx_right = permuteRight[i:i + batch_size]

        left_batch = packPad(paired, lengths, idx_left)
        right_batch = packPad(paired, lengths, idx_right)

        y_batch = (
            torch.tensor(personIDs[idx_left] == personIDs[idx_right])
            .reshape((-1, 1))
            .to(device, dtype=torch.float32)
        )

        output = model2(left_batch, right_batch)
        loss = criterion(output, y_batch)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        epoch_loss += loss.item() * y_batch.size(0)

    # average loss for the epoch
    epoch_loss /= len(permuteLeft)

    if epoch % 10 == 0:
        print(f"Epoch {epoch:3d} – loss: {epoch_loss:.4f}")

model2


Epoch   0 – loss: 0.6905
Epoch  10 – loss: 0.6902
Epoch  20 – loss: 0.6901
Epoch  30 – loss: 0.6900
Epoch  40 – loss: 0.6900
Epoch  50 – loss: 0.6900
Epoch  60 – loss: 0.6900
Epoch  70 – loss: 0.6900
Epoch  80 – loss: 0.6900
Epoch  90 – loss: 0.6900


BigModel(
  (branch): TwoBranch(
    (left): LSTM(3, 4, num_layers=3)
    (right): LSTM(3, 4, num_layers=3)
    (combine): Linear(in_features=8, out_features=8, bias=True)
  )
  (rest): Sequential(
    (0): Linear(in_features=8, out_features=2, bias=True)
    (1): ReLU()
    (2): Linear(in_features=2, out_features=1, bias=True)
    (3): Sigmoid()
  )
)